In [1]:
import pandas as pd
import umap
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc
import sklearn.cluster as cluster

In [2]:
#loading file from Rik's model ouput
df =pd.read_csv('/lustre/scratch117/cellgen/team298/win/df.fil4.GPLVM.csv')

In [3]:
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11
0,UK-CIC10689738+UK-CIC10690026_AAACCTGAGGCAATTA-1,-0.690389,-0.066656,17.737082,2.616001,-3.116531,-1.644330,-2.310348,1.524271,-1.600700,0.676175,-0.503522,0.901487
1,UK-CIC10689738+UK-CIC10690026_AAACCTGCAAGCCGCT-1,-1.037418,0.123800,-3.322257,-5.038774,-3.024778,0.388396,0.617328,-1.411870,0.822993,0.780547,-0.780726,1.128465
2,UK-CIC10689738+UK-CIC10690026_AAACCTGCACGGTTTA-1,1.369259,-0.212231,-3.112305,-5.972466,-3.089788,-0.490101,2.782889,2.182449,-2.940507,3.091495,-0.635181,1.629311
3,UK-CIC10689738+UK-CIC10690026_AAACCTGCATAACCTG-1,-0.382152,0.215146,-0.476123,-4.687519,-1.652420,-2.135794,1.329842,-2.626489,-0.550056,2.213942,-3.195279,-0.504439
4,UK-CIC10689738+UK-CIC10690026_AAACCTGCATCCCACT-1,-0.930730,0.473214,12.831943,0.698647,0.400640,0.953578,8.140420,-4.009368,2.161688,-2.046245,0.112746,0.493701


In [4]:
#load most updated myeloid data with annot4
myeloid = sc.read('/home/jovyan/mounting_point/renal_covid_19_gex_myeloid_soupX_QC_orig.ident_annotated_levle23.h5ad')

In [5]:
myeloid

AnnData object with n_obs × n_vars = 126622 × 1215
    obs: 'annotation22_ES', 'centre', 'sample_id', 'sample_id_broad', 'sample_date', 'sample_date_yr', 'pool', 'pool_broad', 'haniffa_broad_predLabel', 'orig.ident', 'merged_souporcell_cluster', 'merged_souporcell_status', 'patient_id', 'case_control', 'WHO_severity', 'sex', 'calc_age', 'discharge_date', 'date_positive_swab', 'date_first_symptoms', 'admission_date', 'ethnicity', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden_0.8', '5_subcluster', '9_subcluster', 'annotation', 'annotation23_ES', 'annotation24_ES', 'annot3', 'annot2', 'annot1', 'leiden_0.5', 'annot4'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: '5_subcluster_colors', '9_subcluster', '9_subcluster_colors', 'WHO_severity_colors', 'X_pca_harmony_orig.ident', 'annot1_colors', 'annot2_colors', 'annot3_colors', '

In [6]:
#rename df column and make index
df.rename(columns={"Unnamed: 0": 'cell_name'},inplace=True)
df.set_index('cell_name',inplace=True)
df

,0,1,2,3,4,5,6,7,8,9,10,11
cell_name,,,,,,,,,,,,
UK-CIC10689738+UK-CIC10690026_AAACCTGAGGCAATTA-1,-0.690389,-0.066656,17.737082,2.616001,-3.116531,-1.644330,-2.310348,1.524271,-1.600700,0.676175,-0.503522,0.901487
UK-CIC10689738+UK-CIC10690026_AAACCTGCAAGCCGCT-1,-1.037418,0.123800,-3.322257,-5.038774,-3.024778,0.388396,0.617328,-1.411870,0.822993,0.780547,-0.780726,1.128465
UK-CIC10689738+UK-CIC10690026_AAACCTGCACGGTTTA-1,1.369259,-0.212231,-3.112305,-5.972466,-3.089788,-0.490101,2.782889,2.182449,-2.940507,3.091495,-0.635181,1.629311
UK-CIC10689738+UK-CIC10690026_AAACCTGCATAACCTG-1,-0.382152,0.215146,-0.476123,-4.687519,-1.652420,-2.135794,1.329842,-2.626489,-0.550056,2.213942,-3.195279,-0.504439
UK-CIC10689738+UK-CIC10690026_AAACCTGCATCCCACT-1,-0.930730,0.473214,12.831943,0.698647,0.400640,0.953578,8.140420,-4.009368,2.161688,-2.046245,0.112746,0.493701
...,...,...,...,...,...,...,...,...,...,...,...,...
UK-CIC10690384+UK-CIC10690576_TTTGGTTGTTGTCTTT-1,1.792048,0.663562,-0.862063,-4.470762,4.143518,5.666719,-1.938882,0.376993,2.052822,-0.337444,-1.611308,-2.330659
UK-CIC10690384+UK-CIC10690576_TTTGGTTTCTCCTATA-1,0.377639,1.033796,-4.863069,4.595367,2.641854,0.418925,0.003231,3.136955,0.419376,-2.340047,2.373017,-1.556551
UK-CIC10690384+UK-CIC10690576_TTTGGTTTCTGGCGAC-1,0.909309,1.258131,16.236984,3.302101,-0.060014,0.457311,1.339352,-1.379966,1.873648,-4.075781,0.611843,0.856470


In [7]:
#subset df for only myeloid , take the rows of df that have the same index as in myeloid
myeloid_df = df[df.index.isin(myeloid.obs.index)]

In [9]:
myeloid_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
cell_name,,,,,,,,,,,,
UK-CIC10689738+UK-CIC10690026_AAACCTGAGGCAATTA-1,-0.690389,-0.066656,17.737082,2.616001,-3.116531,-1.644330,-2.310348,1.524271,-1.600700,0.676175,-0.503522,0.901487
UK-CIC10689738+UK-CIC10690026_AAACCTGCATCCCACT-1,-0.930730,0.473214,12.831943,0.698647,0.400640,0.953578,8.140420,-4.009368,2.161688,-2.046245,0.112746,0.493701
UK-CIC10689738+UK-CIC10690026_AAACCTGGTACGAAAT-1,-0.962399,0.189368,16.530636,3.066498,-4.318206,-1.444693,-2.513921,2.596895,-0.646221,-0.524370,1.986685,0.513514
UK-CIC10689738+UK-CIC10690026_AAACCTGGTAGCTAAA-1,0.115015,-0.021101,5.062666,-2.562629,6.484043,0.129399,0.026921,1.824796,-0.181073,2.127307,-4.254726,-4.075265
UK-CIC10689738+UK-CIC10690026_AAACCTGTCTGGGCCA-1,-0.643857,-0.242863,14.164966,1.316865,0.062348,-1.680878,6.970035,-4.005841,3.707483,-2.335455,0.479629,0.677857


In [10]:
#getting pseudotime, pseudobatch and latent
myeloid_df_pseudotime = myeloid_df[['0']]
myeloid_df_pseudobatch = myeloid_df[['1']]
myeloid_df_latent = myeloid_df.iloc[:,-10:]

In [13]:
myeloid_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
cell_name,,,,,,,,,,,,
UK-CIC10689738+UK-CIC10690026_AAACCTGAGGCAATTA-1,-0.690389,-0.066656,17.737082,2.616001,-3.116531,-1.644330,-2.310348,1.524271,-1.600700,0.676175,-0.503522,0.901487
UK-CIC10689738+UK-CIC10690026_AAACCTGCATCCCACT-1,-0.930730,0.473214,12.831943,0.698647,0.400640,0.953578,8.140420,-4.009368,2.161688,-2.046245,0.112746,0.493701
UK-CIC10689738+UK-CIC10690026_AAACCTGGTACGAAAT-1,-0.962399,0.189368,16.530636,3.066498,-4.318206,-1.444693,-2.513921,2.596895,-0.646221,-0.524370,1.986685,0.513514
UK-CIC10689738+UK-CIC10690026_AAACCTGGTAGCTAAA-1,0.115015,-0.021101,5.062666,-2.562629,6.484043,0.129399,0.026921,1.824796,-0.181073,2.127307,-4.254726,-4.075265
UK-CIC10689738+UK-CIC10690026_AAACCTGTCTGGGCCA-1,-0.643857,-0.242863,14.164966,1.316865,0.062348,-1.680878,6.970035,-4.005841,3.707483,-2.335455,0.479629,0.677857


In [12]:
myeloid_df_pseudotime.head()

,0
cell_name,
UK-CIC10689738+UK-CIC10690026_AAACCTGAGGCAATTA-1,-0.690389
UK-CIC10689738+UK-CIC10690026_AAACCTGCATCCCACT-1,-0.930730
UK-CIC10689738+UK-CIC10690026_AAACCTGGTACGAAAT-1,-0.962399
UK-CIC10689738+UK-CIC10690026_AAACCTGGTAGCTAAA-1,0.115015
UK-CIC10689738+UK-CIC10690026_AAACCTGTCTGGGCCA-1,-0.643857


In [14]:
#myeloid_df has fewer cells than myeloid as some cells from myeloid are not in myeloid_df
#removing cells from myeloid which are not in myeloid_df
myeloid = myeloid[myeloid.obs_names.isin(myeloid_df.index)]

In [15]:
myeloid

View of AnnData object with n_obs × n_vars = 117863 × 1215
    obs: 'annotation22_ES', 'centre', 'sample_id', 'sample_id_broad', 'sample_date', 'sample_date_yr', 'pool', 'pool_broad', 'haniffa_broad_predLabel', 'orig.ident', 'merged_souporcell_cluster', 'merged_souporcell_status', 'patient_id', 'case_control', 'WHO_severity', 'sex', 'calc_age', 'discharge_date', 'date_positive_swab', 'date_first_symptoms', 'admission_date', 'ethnicity', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden_0.8', '5_subcluster', '9_subcluster', 'annotation', 'annotation23_ES', 'annotation24_ES', 'annot3', 'annot2', 'annot1', 'leiden_0.5', 'annot4'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: '5_subcluster_colors', '9_subcluster', '9_subcluster_colors', 'WHO_severity_colors', 'X_pca_harmony_orig.ident', 'annot2_colors', 'annot4_colors', 'annotatio

In [16]:
myeloid_df.shape

(117863, 12)

In [17]:
#checking if the index are the same
a = 0
for i in range (0,len(myeloid_df)):
    if myeloid.obs.index[i]==myeloid_df.index[i]:
        a = a+1
print (a)
print(len(myeloid.obs))
print (len(myeloid_df))

117863
117863
117863


In [18]:
#adding pseudotime info
myeloid.obs['pseudotime_GPLVM'] = myeloid.obs['calc_age']
myeloid.obs['pseudotime_GPLVM'].update(pd.Series(myeloid_df_pseudotime['0']))

/tmp/ipykernel_6749/2085454226.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  myeloid.obs['pseudotime_GPLVM'] = myeloid.obs['calc_age']
/home/jovyan/my-conda-envs/Py_env2/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


In [19]:
#adding pseudobatch info
myeloid.obs['pseudobatch_GPLVM'] = myeloid.obs['calc_age']
myeloid.obs['pseudobatch_GPLVM'].update(pd.Series(myeloid_df_pseudobatch['1']))

In [22]:
'C141' in list(myeloid.obs['sample_id_broad'].unique())

True

In [23]:
'C116' in list(myeloid.obs['sample_id_broad'].unique())

False

In [24]:
'C139' in list(myeloid.obs['sample_id_broad'].unique())

False

In [25]:
#remove C141 from the samples
myeloid = myeloid[~myeloid.obs['sample_id_broad'].isin(['C141'])]

In [26]:
'C141' in list(myeloid.obs['sample_id_broad'].unique())

False

In [27]:
save_file = '/home/jovyan/scripts/renal_covid_19/scripts_for_publication/monocle_GO/myeloid_for_GPLVM.h5ad'
myeloid.write_h5ad(save_file)

In [28]:
# plotting pseudotime

In [29]:
myeloid

View of AnnData object with n_obs × n_vars = 113501 × 1215
    obs: 'annotation22_ES', 'centre', 'sample_id', 'sample_id_broad', 'sample_date', 'sample_date_yr', 'pool', 'pool_broad', 'haniffa_broad_predLabel', 'orig.ident', 'merged_souporcell_cluster', 'merged_souporcell_status', 'patient_id', 'case_control', 'WHO_severity', 'sex', 'calc_age', 'discharge_date', 'date_positive_swab', 'date_first_symptoms', 'admission_date', 'ethnicity', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden_0.8', '5_subcluster', '9_subcluster', 'annotation', 'annotation23_ES', 'annotation24_ES', 'annot3', 'annot2', 'annot1', 'leiden_0.5', 'annot4', 'pseudotime_GPLVM', 'pseudobatch_GPLVM'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: '5_subcluster_colors', '9_subcluster', '9_subcluster_colors', 'WHO_severity_colors', 'X_pca_harmony_orig.ident', 'an

In [32]:
myeloid.obs['centre'].unique()

['NCL', 'Cambridge']
Categories (2, object): ['Cambridge', 'NCL']

In [33]:
myeloid.obs['case_control'].unique()

['NEGATIVE', 'POSITIVE', 'RECOVERY']
Categories (3, object): ['NEGATIVE', 'POSITIVE', 'RECOVERY']

In [34]:
myeloid.obs['annot4'].unique()

['CD14mono_activated', 'CD16mono', 'pDC', 'CD14mono', 'Int.mono', ..., 'HSPC', 'DC3', 'DC1', 'DC3_IFN', 'ASDC']
Length: 17
Categories (17, object): ['ASDC', 'CD14mono', 'CD14mono_IFN', 'CD14mono_activated', ..., 'Int.mono', 'Platelet', 'RBC', 'pDC']